In [ ]:
!pip install wolframalpha

In [31]:
import json
from sympy.parsing.mathematica import parse_mathematica
from sympy import *

def read_json_file(file_path):
    try:
        with open(file_path, 'r') as json_file:
            data = json.load(json_file)
            return data
    except FileNotFoundError:
        print(f"File '{file_path}' not found.")
        return []

file_path = '/content/derivatives_sample.json'  # Replace with your actual file path
json_data = read_json_file(file_path)

print(json_data)

# Extract the prompts from the dictionary
prompts_list = [item["prompt"] for item in json_data["annotations"]]
prompt_types = [item["type"] for item in json_data["annotations"]]
print(prompts_list, prompt_types)

{'annotations': [{'prompt': 'Find 18 - 11', 'type': 'simple operations'}, {'prompt': 'Find 20 * 4', 'type': 'simple operations'}]}
['Find 18 - 11', 'Find 20 * 4'] ['simple operations', 'simple operations']


# **Factor**

In [ ]:
def factoreq(r):
    plaintext = []
    if(r["queryresult"]["numpods"]>1):
      for k in range(len(r["queryresult"]["pods"])):
        title = r["queryresult"]["pods"][k]["title"]

        if(title == "Factorization over the splitting field"):
          result_data = r["queryresult"]["pods"][k]["subpods"]

          for j in range(len(result_data)):
            plaintext.append(result_data[j]["plaintext"])

    return plaintext

# **Solve three equations**

In [ ]:
def solve_three_quations(r):


    if(r["queryresult"]["numpods"]>1):
      for k in range(len(r["queryresult"]["pods"])):
        title = r["queryresult"]["pods"][k]["title"]

        if(title == "Solution"):
          data = r["queryresult"]["pods"][k]["subpods"][0]
          plaintext = data["plaintext"]

        if(title == "Alternate forms" or title == "Alternate form"):
          alternate_data = r["queryresult"]["pods"][k]["subpods"]

          for j in range(len(alternate_data)):
            alternate_results.append(alternate_data[j]["plaintext"])

    return plaintext, alternate_results

# **Solve for x**

In [ ]:
def solve_for_x(r):
    plaintext = []
    if(r["queryresult"]["numpods"]>1):
      for k in range(len(r["queryresult"]["pods"])):
        title = r["queryresult"]["pods"][k]["title"]

        if(title == "Results"):
          data = r["queryresult"]["pods"][k]["subpods"]

          for j in range(len(data)):
            plaintext.append(data[j]["plaintext"])

    return plaintext

# **Simple Operations**

In [37]:
def simple_operation(r):
    plaintext = []
    if(r["queryresult"]["numpods"]>1):
      for k in range(len(r["queryresult"]["pods"])):
        title = r["queryresult"]["pods"][k]["title"]

        if(title == "Result"):
          data = r["queryresult"]["pods"][k]["subpods"][0]
          plaintext = data["plaintext"]

    return plaintext

## **Wolfram API Results**

In [38]:
import wolframalpha
import requests
import os
import urllib.parse

# App id obtained by the above steps
app_id = 'XTJKXE-UV98JR6EKE'

# Instance of wolf ram alpha
# client class
client = wolframalpha.Client(app_id)

result_list = []
alternate_list = []

for i in range(len(prompts_list)):
  alternate_results = []
  plaintext = []
  prompt = urllib.parse.quote(prompts_list[i])
  prompt_type = prompt_types[i]
  print(prompt)
  query_url = f"http://api.wolframalpha.com/v2/query?" \
            f"appid={app_id}" \
            f"&input={prompt}" \
            f"&format=plaintext" \
            f"&output=json"

  r = requests.get(query_url).json()

  print(f"Result: {r}")
  if prompt_type == "Factor eq":
    result = factoreq(r)
  elif prompt_type == "Solve system of three equations":
    result, alternate_results = solve_three_quations(r)
  elif prompt_type == "Solve for x:":
    result= solve_for_x(r)
  elif prompt_type == "simple operations":
    result = simple_operation(r)

  print(f"Result: {result}")
  print(f"Alternate forms: {alternate_results}")
  alternate_list.append(alternate_results)
  result_list.append( result)


#print(result_list)
#print(alternate_list)

Find%2018%20-%2011
Result: {'queryresult': {'success': True, 'error': False, 'numpods': 5, 'datatypes': 'Math', 'timedout': '', 'timedoutpods': '', 'timing': 0.518, 'parsetiming': 0.209, 'parsetimedout': False, 'recalculate': '', 'id': 'MSP53311a183351cf2f2gic00005g6d7fd33b93d1bh', 'host': 'https://www6b3.wolframalpha.com', 'server': '18', 'related': 'https://www6b3.wolframalpha.com/api/v1/relatedQueries.jsp?id=MSPa53321a183351cf2f2gic0000410h7712712id8bc1697175660060143921', 'version': '2.6', 'inputstring': 'Find 18 - 11', 'pods': [{'title': 'Input', 'scanner': 'Identity', 'id': 'Input', 'position': 100, 'error': False, 'numsubpods': 1, 'subpods': [{'title': '', 'plaintext': '18 - 11'}], 'expressiontypes': {'name': 'Default'}}, {'title': 'Result', 'scanner': 'Simplification', 'id': 'Result', 'position': 200, 'error': False, 'numsubpods': 1, 'primary': True, 'subpods': [{'title': '', 'plaintext': '7'}], 'expressiontypes': {'name': 'Default'}, 'states': [{'name': 'Step-by-step solution'

In [ ]:
# Access the calc_basic results
i=0
for annotation in json_data["annotations"]:
  if(i<100):
    annotation["result"] = str(result_list[i])
    annotation["alternate_forms"] = alternate_list[i]
    i=i+1
print(json_data)
# Save the updated JSON data to a file
with open("calc_basic_result.json", "w") as json_file:
    json.dump(json_data, json_file, indent=4)

print("Updated JSON data saved to 'calc_basic_result.json'")

{'annotations': [{'prompt': 'Solve for x: 1*x^1 + 6*x^2 - 5 = 4*x^1 + 1*x^2 - 7', 'type': 'Solve for x:', 'result': "['x = 1/10 (3 - i sqrt(31))', 'x = 1/10 (3 + i sqrt(31))']", 'alternate_forms': []}, {'prompt': 'Solve for x: 1*x^3 - 4*x^2 + 1 = 1*x^1 + 6*x^2 + 4', 'type': 'Solve for x:', 'result': "['x = 1/3 (10 + (2171/2 - (3 sqrt(38037))/2)^(1/3) + (1/2 (2171 + 3 sqrt(38037)))^(1/3))', 'x = 10/3 - 1/6 (1 - i sqrt(3)) (2171/2 - (3 sqrt(38037))/2)^(1/3) - 1/6 (1 + i sqrt(3)) (1/2 (2171 + 3 sqrt(38037)))^(1/3)', 'x = 10/3 - 1/6 (1 + i sqrt(3)) (2171/2 - (3 sqrt(38037))/2)^(1/3) - 1/6 (1 - i sqrt(3)) (1/2 (2171 + 3 sqrt(38037)))^(1/3)']", 'alternate_forms': []}, {'prompt': 'Solve for x: 2*x^2 - 5*x^1 + 2 = 2*x^3 - 5*x^3 - 2', 'type': 'Solve for x:', 'result': "['x = 1/9 (-2 - 49/(629 - 18 sqrt(858))^(1/3) - (629 - 18 sqrt(858))^(1/3))', 'x = -2/9 + (49 (1 - i sqrt(3)))/(18 (629 - 18 sqrt(858))^(1/3)) + 1/18 (1 + i sqrt(3)) (629 - 18 sqrt(858))^(1/3)', 'x = -2/9 + (49 (1 + i sqrt(3)))/(

In [ ]:
from google.colab import files
files.download('calc_basic_result.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>